In [3]:
import torch

# Auto-detect CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", device)


Using device: cuda


In [6]:
for epoch in range(num_epochs):
    for img_features, input_seq, target_word in dataloader:
        img_features = img_features.to(device)
        input_seq = input_seq.to(device)
        target_word = target_word.to(device)

        optimizer.zero_grad()
        outputs = model(img_features, input_seq)
        loss = criterion(outputs, target_word)
        loss.backward()
        optimizer.step()


NameError: name 'num_epochs' is not defined